In [1]:
%%capture 
!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install gdal
!pip install numpy

In [2]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
import shapely
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import MultiPolygon, Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona


import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

In [18]:
#Here we create a matrix in the 10 by 10 format you want
my_raster = np.ones((10,10))
print(my_raster)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [20]:
#It's just populated with ones. To change a value, use indexing: 
#First number is row, second number is column (count from 0 not 1)
my_raster[0,2] = 9
print(my_raster)

[[1. 1. 9. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [33]:
#If you want, you can also manually edit the array 

my_raster = np.array(
[[10., 9., 8., 7., 6., 5., 4., 3., 2., 1.], #Here, I have edited the row
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], #You can do that here too by just replacing the numbers
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], #If you are editing like this, the commas need to be there 
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
print(my_raster)

[[10.  9.  8.  7.  6.  5.  4.  3.  2.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]


In [21]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/

dirname = '/content/drive/MyDrive/'

Mounted at /content/drive
/content/drive/MyDrive


In [28]:
#Now we can output the array to a tiff file :) 
#If you go to the recent folder in your google drive, you will see it there 

driver = gdal.GetDriverByName( 'GTiff' )
out_filename = 'my_raster.tif'
dst_ds=driver.Create(out_filename,10,10,1,gdal.GDT_Byte)
#Next, we set the location of the upper left corner of the raster 
#I've made it so it's right beside City Hall in Toronto, you can change it
#The first number is the UTM Zone 17 East coordinate, second number is the North 
#The two 15s refer to the size of each pixel (15m), if you change to another number
#Make sure one of them is still negative (second one!!) - important 
dst_ds.SetGeoTransform([630380.229, 15, 0, 4834628.614, 0, -15 ]) 
srs = osr.SpatialReference()
srs.SetUTM( 17, 1 ) #Set UTM Zone 17 (you will have to look up which one you need)
srs.SetWellKnownGeogCS( 'WGS84' ) #Set coordinate system to WGS84 
dst_ds.SetProjection( srs.ExportToWkt() )
dst_ds.GetRasterBand(1).WriteArray( my_raster ) #Write to drive 

0